<a href="https://colab.research.google.com/github/JHyunjun/torch_GAN_AutoEncoder/blob/main/ANN_W-GAN_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import array
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt') 
#df_train = df_train.astype(int)
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data_input = df_train.iloc[:,:-1]
train_data_output = df_train.iloc[:,-1]
test_data_input = df_test.iloc[:,:-1]
test_data_output = df_test.iloc[:,-1]

train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]

trainSet_for_result = train_data
testSet_for_result = test_data

print("train data input : ",train_data_input.shape,train_data_input)
print("train data output : ",train_data_output.shape, train_data_output)
print("train_data : ",train_data.shape,train_data)
print("test_data : ",test_data.shape,test_data)

In [ ]:
time1 = 5500
time2 = 5800
train_gan = df_train.iloc[time1:time2, :].values
print(train_gan.shape)

train_gan2 = df_test.iloc[time1:time2, :].values
#train_gan2 = train_gan2.to_numpy()
print(train_gan)

In [ ]:
#Normalization

def origin_minmax(data) : 
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])
  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i]) 
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(train_gan)
arr_max2, arr_min2 = origin_minmax(train_gan2)

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

train_gan = MinMaxScaler(train_gan)
train_gan2 = MinMaxScaler(train_gan2)

def back_MinMax(data,max,min):
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (max[i] - min[i]) + min[i]
    pass
  return data

train_gan = torch.tensor([train_gan], dtype = torch.float32)
train_gan2 = torch.tensor([train_gan2], dtype = torch.float32)
print(train_gan.shape)

print(arr_max1)
print(arr_min1)
print(arr_max2)
print(arr_min2)

In [ ]:
train_gan = train_gan.transpose(1,2)
train_gan2 = train_gan2.transpose(1,2)

print(train_gan.shape) 
print("train_gan.shape[0] : ",train_gan.shape[0])
print("train_gan.shape[1] : ",train_gan.shape[1])

class Discriminator(nn.Module):
    
    def __init__(self):
        # initialise parent pytorch class
        super().__init__()
        self.a = nn.Linear(train_gan.shape[1], 200)
        self.b = nn.LeakyReLU(0.02)

        self.c = nn.Linear(200, 400)
        self.d = nn.LeakyReLU(0.02)
        #self.e = nn.LayerNorm(Data_seq * dim_size)

        self.f = nn.Linear(400, 1) 
        self.g = nn.Sigmoid()
        
        # create loss function
        self.loss_function = nn.BCELoss()

        # create optimiser, simple stochastic gradient descent
        #self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)
        # Wasserstain
        self.optimiser = torch.optim.RMSprop(self.parameters(), lr = 5e-5)

        # counter and accumulator for progress
        self.counter = 0;
        self.progress = []
        pass
    
    def forward(self, inputs):
        #simply run model
        x = self.a(inputs)
        #print("A")
        x = self.b(x)
        #print("B")
        x = self.c(x)
        #print("C")
        x = self.d(x)
        #print("D")
        x = self.f(x)
        #print("E")
        x = self.g(x)
        #print("F")

        return x

    def train(self, inputs, targets, gens):
        # calculate the output of the network
        #print("Start")
        outputs = self.forward(inputs)
        #print("After D_self.forward(inputs)")
        targets_w = self.forward(gens)
        #print("After self.forward(gens)")

        # calculate loss
        #loss = self.loss_function(outputs, targets)
        #print("After Calculating D loss")

        # Wasserstein loss
        loss = -(torch.mean(outputs) - torch.mean(targets_w)) - torch.mean(inputs - gens)

        # increase counter and accumulate error every 10
        self.counter += 1;
        if (self.counter % 10 == 0):
            self.progress.append(loss.item())
            pass
        if (self.counter % 1000 == 0):
            print("counter = ", self.counter)
            pass

        # zero gradients, perform a backward pass, update weights
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
        
        #W-GAN
        for p in self.parameters() : 
          p.data.clamp_(-0.01,0.01)
        pass
    
    def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Discriminator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        pass
    
    pass

In [ ]:
#Training the Discriminator
N = 10  #iterations

D = Discriminator()
targets = torch.FloatTensor([1.0])
non_target = torch.FloatTensor([0.0])

In [ ]:
#Checking the performance of Discriminator
#D.plot_progress()


class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,

    def forward(self, x):
        return x.view(*self.shape)


In [ ]:
# Constructing Generator for AutoEncoder Structure

#latent_space = 220
Gen_nodes = 100 # *4

class Generator(nn.Module) : 
  def __init__(self) : 
    super().__init__()

    self.a = nn.Linear(train_gan.shape[1], Gen_nodes*4)
    self.b = nn.LeakyReLU(0.02)
    self.c = nn.LayerNorm(Gen_nodes*4)

    self.d = nn.Linear(Gen_nodes*4,Gen_nodes*4)
    self.e = nn.LeakyReLU(0.02)
    self.f = nn.LayerNorm(Gen_nodes * 4)

    self.g = nn.Linear(Gen_nodes * 4, train_gan.shape[1])
    self.h = nn.Sigmoid()
        #View((1, train_gan.shape[1], 1))   
    
    

    #self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)
    #Wasserstain
    self.optimiser = torch.optim.RMSprop(self.parameters(), lr = 5e-5)
    self.counter = 0
    self.progress = []

    pass

  def forward(self, inputs) : 
    #print("inputs. shape : ",inputs.shape) #[1,5,1] 끝남.
    x = self.a(inputs)
    #print("a. shape : ",x.shape)
    x = self.b(x)
    #print("b. shape : ",x.shape)
    x = self.c(x)
    #print("c. shape : ",x.shape)
    x = self.d(x)
    #print("d. shape : ",x.shape)
    x = self.e(x) 
    #print("e. shape : ",x.shape)
    x = self.f(x) 
    #print("f. shape : ",x.shape)
    x = self.g(x)
    #print("g. shape : ",x.shape)
    x = self.h(x) #Sigmoid
    #x = x.reshape(1, train_gan.shape[1] , train_gan.shape[2]) # [1,2,600]
    return x

  def train(self, D, inputs, targets) : 
    g_output = self.forward(inputs)
    d_output = D.forward(g_output)
    #loss = D.forward(g_output)
    #loss = D.loss_function(d_output, targets)

    #Wasserstain
    loss = -torch.mean(d_output)

    self.counter+=1;
    if (self.counter % 10 == 0) :
      self.progress.append(loss.item())
      pass

    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()

  def plot_progress(self):
        df = pd.DataFrame(self.progress, columns=['Generator loss'])
        df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0))
        
  pass

In [ ]:
ref = train_gan[:,:,:] #Normalizing된 신호
print(ref.shape)

In [ ]:
#Training both Generator and Discriminator for pattern
D = Discriminator()
G = Generator()

for x in range(10) : 
  for i in range(time2-time1) : #i는 0부터 시작
    refs = train_gan[:,:,(i):(i+1)] #Normalizing된 신호
    refs = np.squeeze(refs) # [1,5,1] -> [5], 1차원짜리 다 없앰
    for j in range(1) : 
      D.train(refs, torch.FloatTensor([1.0]),G.forward(refs))
      G.train(D, refs, torch.FloatTensor([1.0]))
    print("i iteration : ", i)

  pass
pass
pass

In [ ]:
#Plotting the Generator Loss
D.plot_progress()
G.plot_progress()
print(len(train_gan)) 

In [ ]:
for i in range(time2-time1) : 
  seed = train_gan[:,:, i:(i+1)]
  seed = np.squeeze(seed)
  out = G.forward(seed)
  out = out.detach().cpu().numpy()
  #print(out.shape)
  out = np.reshape(out, [1,train_gan.shape[1]])
  seed = np.reshape(seed, [1,train_gan.shape[1]])
  #print(out.shape)
  out_dim = back_MinMax(out, arr_max1, arr_min1)
  print("Generated Value : ",i," = ", out_dim)
  real_value = back_MinMax(seed, arr_max1, arr_min1)
  print("Real Value : ",i, " = ",real_value )
  print("                            ")